# Some tests of convergence for electron trace wrt dt

## Some of these runs are slow ~20 minutes. Not useful for paramater scans, but indicate that ~10 steps per orbit is sufficient

In [ ]:
%run config.py

In [ ]:
from ford1991 import solve_3d
from scipy.constants import c, electron_mass as me, elementary_charge as qe, mu_0 as mu0
from qtnm_fields import Coil_Field, Bath_Tub_Field

In [ ]:
# Deflection from beam direction in degrees
theta_deg = -1
theta = np.deg2rad(theta_deg)
# Initial kinetic energy (eV)
T = 18600
# Rel. gamma
gamma_rel = T * qe / (me*c**2) + 1
# (v/c)^2
beta_sq = 1 - 1 / gamma_rel**2
# Initial electron speed
v0 = np.sqrt(beta_sq) * c
# Background B-field (T)
B0 = np.array([0, 0, 1.0])
# Switch off Larmor term
tau = 0.0

In [ ]:
# Initial conditions for the electron
ic = [0, 0, 0, v0 * np.cos(theta), 0, v0 * np.sin(theta), 0.0]

In [ ]:
# Set up coil calculation. Coil in X-Y plane, centred on (0,0)
Ny = 51
Nz = 101
Ntheta = 101
Rcoil = 0.005
I = 40

coil = Coil_Field(Ntheta, R=Rcoil, I=I)

# Mesh for plotting field
ygrid = np.linspace(-0.025,0.025,Ny)
zgrid = np.linspace(-0.025,0.025,Nz)

Y, Z = np.meshgrid(ygrid, zgrid)

In [ ]:
bx = np.zeros_like(Y)
by = np.zeros_like(Y)
bz = np.zeros_like(Y)

for i in range(Nz):
    for j in range(Ny):
        x = 0.0
        y = ygrid[j]
        z = zgrid[i]
        
        bx[i,j], by[i,j], bz[i,j] = coil.evaluate_field_at_point(x, y, z)

In [ ]:
# Position of two coils
zc1 = -0.01
zc2 = 0.01

bath = Bath_Tub_Field(Ntheta, R=Rcoil, I=I, Z1 = zc1, Z2 = zc2)

In [ ]:
for i in range(Nz):
    for j in range(Ny):
        x = 0.0
        y = ygrid[j]
        z = zgrid[i]
        
        bx[i,j], by[i,j], bz[i,j] = bath.evaluate_field_at_point(x, y, z)

In [ ]:
%%time
# This is -slow- takes around 20 minutes
qtnm_bottle = Bath_Tub_Field(Ntheta, R=Rcoil, I=I, Z1 = zc1, Z2 = zc2, background=B0)
res1 = solve_3d(300, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau,
                calc_b_field=qtnm_bottle.evaluate_field_at_point, ic=ic)
print('Final simulated time = %.4Es' % res1.t[-1])

In [ ]:
%%time
# Let's check if a higher CFL number can be used
# CFL defined here such that we have 1 / CFL time-steps per orbit
res2 = solve_3d(300, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau,
                calc_b_field=qtnm_bottle.evaluate_field_at_point, ic=ic, cfl=1e-2)
print('Final simulated time = %.4Es' % res2.t[-1])

In [ ]:
# Compare the vertical motion of the two runs
plt.plot(res1.t, res1.y[2], label='CFL = 1e-3')
plt.plot(res2.t, res2.y[2], linestyle='--', label='CFL = 1e-2')
plt.legend()

## Seems identical. Drop CFL to 0.01?

## Let's set-up a different trap, that doesn't restrict the electrons to the beam region

In [ ]:
zc1 = -0.03
zc2 = 0.03
qtnm_bottle = Bath_Tub_Field(Ntheta, R=Rcoil, I=I, Z1 = zc1, Z2 = zc2, background=B0)

In [ ]:
%%time
res3 = solve_3d(300, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau,
                calc_b_field=qtnm_bottle.evaluate_field_at_point, ic=ic, cfl=1e-2)
print('Final simulated time = %.4Es' % res3.t[-1])

In [ ]:
# Need a larger mesh to plot the field for

ygrid = np.linspace(-0.05,0.05,Ny)
zgrid = np.linspace(-0.05,0.05,Nz)
Y, Z = np.meshgrid(ygrid, zgrid)


for i in range(Nz):
    for j in range(Ny):
        x = 0.0
        y = ygrid[j]
        z = zgrid[i]
        
        bx[i,j], by[i,j], bz[i,j] = qtnm_bottle.evaluate_field_at_point(x, y, z) - B0

x = res3.y[0]
y = res3.y[1]
z = res3.y[2]

plt.streamplot(Y, Z, by, bz, color="blue", linewidth=0.1, density=2)
plt.plot(-Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(-Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.axhspan(-0.005, 0.005, color='grey', alpha=0.5)
ax = plt.gca()
t2 = ax.text(
    -0.0325, 0, "Beam", ha="center", va="center", rotation=0, size=15, color='black',
    bbox=dict(boxstyle="rarrow,pad=0.3", fc="cyan", ec="cyan", lw=2,alpha=1.0))

plt.yticks([-0.015,-0.05,0.05,0.015])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

xtrace = x[::2000]
ztrace = z[::2000]
plt.plot(xtrace, ztrace, alpha = 0.25)

plt.xlim(ygrid[0], ygrid[-1])
plt.ylim(zgrid[0], zgrid[-1])
plt.tight_layout()

In [ ]:
# Check what sort of range of vertical field the electron has experienced
plt.plot(zgrid[:], bz[:,25])
plt.axvspan(res3.y[2][0], res3.y[2][-1], color='blue', alpha=0.5)

In [ ]:
%%time

# Let's run for longer, and check the electron was actually trapped.
# Again, this is -slow- ~ 20 minutes
res4 = solve_3d(3000, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau,
                calc_b_field=qtnm_bottle.evaluate_field_at_point, ic=ic, cfl=1e-2)
print('Final simulated time = %.4Es' % res4.t[-1])

In [ ]:
x = res4.y[0]
y = res4.y[1]
z = res4.y[2]

plt.streamplot(Y, Z, by, bz, color="blue", linewidth=0.1, density=2)
plt.plot(-Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(-Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.axhspan(-0.005, 0.005, color='grey', alpha=0.5)
ax = plt.gca()
t2 = ax.text(
    -0.0325, 0, "Beam", ha="center", va="center", rotation=0, size=15, color='black',
    bbox=dict(boxstyle="rarrow,pad=0.3", fc="cyan", ec="cyan", lw=2,alpha=1.0))

plt.yticks([-0.015,-0.05,0.05,0.015])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

xtrace = x[::2000]
ztrace = z[::2000]
plt.plot(xtrace, ztrace, alpha = 0.25)

plt.xlim(ygrid[0], ygrid[-1])
plt.ylim(zgrid[0], zgrid[-1])
plt.tight_layout()

In [ ]:
%%time
## How bad can we go with the CFL number? 10 time steps per orbit?
res5 = solve_3d(3000, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau,
                calc_b_field=qtnm_bottle.evaluate_field_at_point, ic=ic, cfl=1e-1)
print('Final simulated time = %.4Es' % res5.t[-1])

In [ ]:
# Seems not terrible?
plt.plot(res4.t, res4.y[2])
plt.plot(res5.t, res5.y[2], linestyle='--')